# SFC, FC & SC - LOBE vs HCP #

In [ ]:
subjects=snakemake.params.subjects

for dataset in subjects.keys():
    subjects[dataset] = [ 'sub-'+subject for subject in subjects[dataset] ]

In [ ]:
subjects

## Importing packages and setting paths ##

In [ ]:
import pandas as pd
import nibabel as nib
from glob import glob
import numpy as np


In [ ]:
#first load up the label dseg table
atlas='brainnetome'

df_labels = pd.read_csv(f'resources/atlas/atlas-{atlas}_dseg.tsv',sep='\t')
df_labels

df_network_names = df_labels.networks.unique()
df_network_names

df_tabular = pd.read_csv(snakemake.input.tsv,sep='\t')
#df_tabular = pd.read_csv('results/analysis/df_tabular.csv',sep='\t')

df_tabular



In [ ]:
# Schaefer Patterns
#fc_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-91k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_bold.pconn.nii'
#sc_pattern='results/{dataset}/sub-{subject}/dwi/sub-{subject}_den-91k_atlas-schaefer_struc.pconn.nii'
#sfc_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-91k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_sfc.pscalar.nii'

# Brainnetome Patterns
# Schaefer Patterns
fc_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-91k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-brainnetome_bold.pconn.nii'
sc_pattern='results/{dataset}/sub-{subject}/dwi/sub-{subject}_den-91k_atlas-brainnetome_struc.pconn.nii'
sfc_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-91k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-brainnetome_sfc.pscalar.nii'




In [ ]:
sfc_paths={}
sfc_paths['HCP'] = sorted(glob(sfc_pattern.format(dataset='HCP',subject='*')))
sfc_paths['LOBE'] = sorted(glob(sfc_pattern.format(dataset='LOBE',subject='*')))

fc_paths={}
fc_paths['HCP'] = sorted(glob(fc_pattern.format(dataset='HCP',subject='*')))
fc_paths['LOBE'] = sorted(glob(fc_pattern.format(dataset='LOBE',subject='*')))

sc_paths={}
sc_paths['HCP'] = sorted(glob(sc_pattern.format(dataset='HCP',subject='*')))
sc_paths['LOBE'] = sorted(glob(sc_pattern.format(dataset='LOBE',subject='*')))

#print(sfc_paths)
#print(fc_paths)
#print(sc_paths)

In [ ]:
#from snakebids.utils.snakemake_io import glob_wildcards
#subjects = glob_wildcards(sfc_pattern.format(dataset='HCP',subject='{subject}'))


In [ ]:
sfc={}
fc={}
sc={}
num_parcels = nib.load(sfc_paths['LOBE'][0]).get_fdata().shape[1]
datasets=['HCP','LOBE']
for dataset in datasets:
    sfc[dataset] = np.concatenate(  [ nib.load(path).get_fdata()  for path in sfc_paths[dataset]] ,axis=0)
    fc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in fc_paths[dataset]] ,axis=0)
    sc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in sc_paths[dataset]] ,axis=0)

In [ ]:
df_tabular

In [ ]:

#this adds RSFC features as new columns to the df_tabular

df_list=[]
for dataset in datasets:
    df_=pd.DataFrame(sfc[dataset],
                 columns=('RSFC_'+df_labels.name))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    df_['participant_id'] = snakemake.params.subjects[dataset]
    
    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

## Average FC per parcel - group difference ##

In [ ]:
#set diagonal to be nan
x1 = fc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = fc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

In [ ]:
x1.shape

In [ ]:
x1 = np.nanmean(x1,axis=2)
x2 = np.nanmean(x2,axis=2)

In [ ]:
x1.shape

In [ ]:
x2.shape

In [ ]:
#add FC parcel avgs to tabular df

data_={'HCP':x1,'LOBE':x2}

df_list=[]
for dataset in datasets:

    
    df_=pd.DataFrame(data_[dataset],
                 columns=('RFC_'+df_labels.name))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    df_['participant_id'] = snakemake.params.subjects[dataset]

    
    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

# Structural Connectivity #

## Average SC per parcel - group difference ##

In [ ]:
#set diagonal to be nan
x1 = sc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = sc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

In [ ]:
x1.shape

In [ ]:
x1 = np.nanmean(x1,axis=2)
x1.shape
x2 = np.nanmean(x2,axis=2)
x2.shape

In [ ]:
x1.shape

In [ ]:
x2.shape

In [ ]:
#add SC parcel avgs to tabular df

data_={'HCP':x1,'LOBE':x2}

df_list=[]
for dataset in datasets:

    
    df_=pd.DataFrame(data_[dataset],
                 columns=('RSC_'+df_labels.name))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    df_['participant_id'] = snakemake.params.subjects[dataset]

    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

In [ ]:
df_tabular

# Network-Wise, LOBE vs HCP #

In [ ]:
# Schaefer patterns
#fc_network_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_netbold.pconn.nii'
#sc_network_pattern='results/{dataset}/sub-{subject}/dwi/sub-{subject}_den-32k_atlas-schaefer_netstruc.pconn.nii'
#sfc_network_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_netsfc.pconn.nii'

# Brainnetome Patterns
fc_network_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-91k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-brainnetome_netbold.pconn.nii'
sc_network_pattern='results/{dataset}/sub-{subject}/dwi/sub-{subject}_den-91k_atlas-brainnetome_netstruc.pconn.nii'
sfc_network_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-91k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-brainnetome_netsfc.pconn.nii'

datasets=['HCP','LOBE']

In [ ]:
sfc_netpaths={}
sfc_netpaths['HCP'] = sorted(glob(sfc_network_pattern.format(dataset='HCP',subject='*')))
sfc_netpaths['LOBE'] = sorted(glob(sfc_network_pattern.format(dataset='LOBE',subject='*')))

fc_netpaths={}
fc_netpaths['HCP'] = sorted(glob(fc_network_pattern.format(dataset='HCP',subject='*')))
fc_netpaths['LOBE'] = sorted(glob(fc_network_pattern.format(dataset='LOBE',subject='*')))

sc_netpaths={}
sc_netpaths['HCP'] = sorted(glob(sc_network_pattern.format(dataset='HCP',subject='*')))
sc_netpaths['LOBE'] = sorted(glob(sc_network_pattern.format(dataset='LOBE',subject='*')))

#print(sfc_netpaths)
#print(fc_netpaths)
#print(sc_netpaths)

In [ ]:
sfc={}
fc={}
sc={}
num_parcels = nib.load(sfc_netpaths['LOBE'][0]).get_fdata().shape[1]

for dataset in datasets:

    sfc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in sfc_netpaths[dataset]] ,axis=0)  
    #for path in sfc_netpaths[dataset]] ,axis=0)
    fc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in fc_netpaths[dataset]] ,axis=0)
    sc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in sc_netpaths[dataset]] ,axis=0)

## SFC ##

In [ ]:
sfc['LOBE'].shape

### Average per Network - group difference ###

In [ ]:
#set diagonal to be nan
x1 = sfc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = sfc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

In [ ]:
x1.shape

In [ ]:
x1 = np.nanmean(x1,axis=2)
x2 = np.nanmean(x2,axis=2)

In [ ]:
x1.shape

In [ ]:
x1

In [ ]:
x2.shape

In [ ]:
#add SFC network avgs to tabular df

data_={'HCP':x1,'LOBE':x2}

df_list=[]
for dataset in datasets:

    
    df_=pd.DataFrame(data_[dataset],
                 columns=('NSFC_'+df_network_names))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    df_['participant_id'] = snakemake.params.subjects[dataset]

    
    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

In [ ]:
print(df_tabular)

## Functional Connectivity ##

In [ ]:
#set diagonal to be nan
x1 = fc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = fc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

### Average FC per network - group difference ###

In [ ]:
x1 = np.nanmean(x1,axis=2)
x2 = np.nanmean(x2,axis=2)

In [ ]:
x2.shape

In [ ]:
#add FC network avgs to tabular df

data_={'HCP':x1,'LOBE':x2}

df_list=[]
for dataset in datasets:

    
    df_=pd.DataFrame(data_[dataset],
                 columns=('NFC_'+df_network_names))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    df_['participant_id'] = snakemake.params.subjects[dataset]

    
    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

In [ ]:
df_tabular

## Structural Connectivity ##

In [ ]:
#set diagonal to be nan
x1 = sc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = sc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

### Average SC per network - group differences ###

In [ ]:
x1 = np.nanmean(x1,axis=2)
x2 = np.nanmean(x2,axis=2)

In [ ]:
x2.shape

In [ ]:
#add SC network avgs to tabular df

data_={'HCP':x1,'LOBE':x2}

df_list=[]
for dataset in datasets:

    
    df_=pd.DataFrame(data_[dataset],
                 columns=('NSC_'+df_network_names))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    df_['participant_id'] = snakemake.params.subjects[dataset]

    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

In [ ]:
print(df_tabular)

### Save regional and network SFC, SC & FC to masterlist ###

In [ ]:
df_tabular.to_csv(snakemake.output.tsv,sep='\t',index=False)


df_tabular 